flip class (EP 6-2)
2022-03-25

In [ ]:
from tensorflow.keras.datasets import imdb
import numpy as np

데이터 로딩

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print(train_data)
print(train_labels)

단어와 정수index 를 매핑한 딕셔너리 +
정수index 와 단어를 매핑하도록 reverse

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index)
decoded_review = " ".join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
print(decoded_review)

One-Hot encoding (vectorize)
for MNIST

In [ ]:
def to_one_hot(labels, dimension=10):
  results = np.zeros((len(labels), dimension))
  for i, label in enumerate(labels):
    results[i, label] = 1
  return results

vectorize_sequences

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results

useage of v_s

In [ ]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Model definition

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers, losses, metrics

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')) # output = 0 or 1

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


slide. 13

실제 학습

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(partial_x_train, partial_y_train,
                    epochs=20, batch_size=512,
                    validation_data=(x_val, y_val))

results = model.evaluate(x_test, y_test)
print(results)
model.predict(x_test)

multiple?

In [ ]:
from tensorflow.keras.datasets import reuters

(train_data, train_labesl), (test_data, test_labels) = reuters.load_data(num_words=10000)

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

from tensorflow.keras.utils import to_categorical
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

model

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

실제 학습

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

history = model.fit(partial_x_train, partial_y_train,
                    epochs=20, batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=1)])

results = model.evaluate(x_test, one_hot_test_labels)
print(results)

Comparing with the random baseline

In [ ]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
accuracy = float(np.sum(hits_array)) / len(test_labels)
print(accuracy)

results = model.evaluate(x_test, one_hot_test_labels)
print(results)

prediction

In [ ]:
predictions = model.predict(x_test)
print(np.argmax(predictions[0]))
print(predictions)